# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [47]:
//DONE

val housing = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [48]:
// DONE

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [49]:
// DONE

housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [50]:
// DONE

housing.take(5)

Array([-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY], [-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY], [-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY], [-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY], [-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY])

Print the number of records with population more than 10000.

In [51]:
// DONE

housing.filter("population > 10000").count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [52]:
// DONE

housing.select("housing_median_age", "total_rooms", "median_house_value", "population").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [53]:
// DONE

import org.apache.spark.sql.functions._

housing.select(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [54]:
// DONE

housing.groupBy("ocean_proximity").count().sort(desc("count")).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [55]:
// DONE

housing.groupBy("ocean_proximity").agg(avg("median_house_value").alias("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [56]:
// DONE

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value "+
           "FROM df "+
           "GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [57]:
// DONE

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().
             setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).
             setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

va = vecAssembler_263c117ff776
housingAttrs = [longitude: double, latitude: double ... 9 more fields]


[longitude: double, latitude: double ... 9 more fields]

In [58]:
// DONE

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head()

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


coeff = 


1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [59]:
// DONE

val housingCol1 = housing.withColumn("rooms_per_household", housing("total_rooms")/housing("households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", housing("total_bedrooms")/housing("total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", housing("population")/housing("households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1 = [longitude: double, latitude: double ... 9 more fields]
housingCol2 = [longitude: double, latitude: double ... 10 more fields]
housingExtra = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [60]:
// DONE

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [61]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel = label
colCat = ocean_proximity
colNum = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [62]:
// DONE

for (c <- colNum) {
    val count = renamedHousing.filter(renamedHousing(c).isNull || renamedHousing(c)==="").count
    println(s"Missing values in ${c}: ${count}")
}

Missing values in longitude: 0
Missing values in latitude: 0
Missing values in housing_median_age: 0
Missing values in total_rooms: 0
Missing values in total_bedrooms: 207
Missing values in population: 0
Missing values in households: 0
Missing values in median_income: 0
Missing values in rooms_per_household: 0
Missing values in bedrooms_per_room: 207
Missing values in population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [63]:
// DONE

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").
                            setInputCols(Array("total_bedrooms", "bedrooms_per_room")).
                            setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))

val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)
imputedHousing.select("*").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129

imputer = imputer_ccbbd4feb679
imputedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [64]:
// DONE

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("feature_scaling")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("feature_scaling").setOutputCol("scaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|     feature_scaling|     scaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

va = vecAssembler_fd7a348cb18a
featuredHousing = [longitude: double, latitude: double ... 12 more fields]
scaler = stdScal_cd8296e0bd43
scaledHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [65]:
// DONE

renamedHousing.select(countDistinct("ocean_proximity")).show

+-------------------------------+
|count(DISTINCT ocean_proximity)|
+-------------------------------+
|                              5|
+-------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [66]:
// DONE

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("indexed_ocean_proximity")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2

indexer = strIdx_33afeb4d81b4
idxHousing = [longitude: double, latitude: double ... 12 more fields]


[longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [67]:
indexer.fit(renamedHousing).labels

Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [68]:
// DONE

import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().
                  setInputCols(Array("indexed_ocean_proximity")).
                  setOutputCols(Array("encoded_ocean_proximity"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21

encoder = oneHotEncoder_5b2536389e33
ohHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [69]:
// DONE

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))

val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|     feature_scaling|     scaled_features|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14

numPipeline = pipeline_42c725549de9
catPipeline = pipeline_7a8fa82ebe0a
pipeline = pipeline_b3b255323a24
newHousing = [longitude: double, latitude: double ... 15 more fields]


[longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [70]:
// DONE

val va2 = new VectorAssembler().setInputCols(Array("scaled_features", "encoded_ocean_proximity")).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



va2 = vecAssembler_1cdba3a449ae
dataset = [features: vector, label: double]


[features: vector, label: double]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [25]:
// DONE

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet = [features: vector, label: double]
testSet = [features: vector, label: double]


[features: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [26]:
// DONE

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setFeaturesCol("features").setLabelCol("label")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-57923.96917093824,-58826.19295840977,13131.217303085408,8365.53442507742,1277.8992855851639,-52109.41740237342,46046.832562465934,76814.23235563966,7895.693751287305,16593.76632157703,1368.6472847362743,-119760.46064096155,-153570.11396991325,-115224.28648730584,-124265.82527482923], Intercept: -2409967.3096547695
RMSE: 67667.98324348583


lr = linReg_8ed7273bedf8
lrModel = linReg_8ed7273bedf8
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@116a17e8


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@116a17e8

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [27]:
// DONE

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 202151.0004242831|94600.0|[-62.065401082150...|
|104471.82985634916|85800.0|[-62.040445150874...|
|179074.50183627475|79000.0|[-62.025471592109...|
| 189182.4822782632|90100.0|[-61.985542102068...|
|170982.93866956607|69000.0|[-61.985542102068...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68964.25632590684


predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_72e568e71bd3
rmse = 68964.25632590684


68964.25632590684

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [28]:
// DONE

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|176167.91061452514|94600.0|[-62.065401082150...|
|144890.00261437907|85800.0|[-62.040445150874...|
|144890.00261437907|79000.0|[-62.025471592109...|
|199780.22157996148|90100.0|[-61.985542102068...|
| 154789.1402714932|69000.0|[-61.985542102068...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67053.39913413515


dt = dtr_c38613883607
dtModel = DecisionTreeRegressionModel (uid=dtr_c38613883607) of depth 5 with 63 nodes
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_dd428e052c8d
rmse = 67053.39913413515


67053.39913413515

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [29]:
// DONE

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|203072.81350434362|94600.0|[-62.065401082150...|
|148987.44520436606|85800.0|[-62.040445150874...|
| 149551.8618560081|79000.0|[-62.025471592109...|
|205201.90189195206|90100.0|[-61.985542102068...|
|151961.67065403718|69000.0|[-61.985542102068...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 64710.15500594249


rf = rfr_77df566cec0d
rfModel = RandomForestRegressionModel (uid=rfr_77df566cec0d) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_f4dcba2ec6da
rmse = 64710.15500594249


64710.15500594249

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [30]:
// DONE

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|134316.50598216953|94600.0|[-62.065401082150...|
| 92553.31963846851|85800.0|[-62.040445150874...|
| 87337.41396532452|79000.0|[-62.025471592109...|
| 76583.48795242082|90100.0|[-61.985542102068...|
| 84454.79678027178|69000.0|[-61.985542102068...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 55696.3249933125


gb = gbtr_ce010945afc7
gbModel = GBTRegressionModel (uid=gbtr_ce010945afc7) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_e55b8aa1d694
rmse = 55696.3249933125


55696.3249933125

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [31]:
// DONE

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
   .addGrid((rfModel).numTrees, Array(1, 5, 10))
   .addGrid(rfModel.maxDepth, Array(5, 10, 15))
   .build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
|135366.04761904763|94600.0|[-62.065401082150...|
| 78539.87403993856|85800.0|[-62.040445150874...|
|100892.42424242424|79000.0|[-62.025471592109...|
|120664.70588235294|90100.0|[-61.985542102068...|
| 122060.7575757576|69000.0|[-61.985542102068...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 52350.97753533557


paramGrid = 


Array({
	rfr_77df566cec0d-maxDepth: 5,
	rfr_77df566cec0d-numTrees: 1
}, {
	rfr_77df566cec0d-maxDepth: 5,
	rfr_77df566cec0d-numTrees: 5
}, {
	rfr_77df566cec0d-maxDepth: 5,
	rfr_77df566cec0d-numTrees: 10
}, {
	rfr_77df566cec0d-maxDepth: 10,
	rfr_77df566cec0d-numTrees: 1
}, {
	rfr_77df566cec0d-maxDepth: 10,
	rfr_77df566cec0d-numTrees: 5
}, {
	rfr_77df566cec0d-maxDepth: 10,
	rfr_77df566cec0d-numTrees: 10
}, {
	rfr_77df566cec0d-maxDepth: 15,
	rfr_77df566cec0d-numTrees: 1
}, {
	rfr_77df566cec0d-maxDepth: 15,
	rfr_77df566cec0d-numTrees: 5
}, {
	rfr_77df566cec0d-maxDepth: 15,
	rfr_77df...


---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [32]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType, StringType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "Input column 1")
    final val inputCol2 = new Param[String](this, "inputCol2", "Input column 2")
    final val outputCol = new Param[String](this, "outputCol", "Output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    val idx1 = schema.fieldIndex($(inputCol1))
    val field1 = schema.fields(idx1)
    val idx2 = schema.fieldIndex($(inputCol2))
    val field2 = schema.fields(idx2)
    if (field1.dataType != StringType && field2.dataType != StringType) {
      throw new Exception(s"Input type ${field1.dataType} or ${field2.dataType} did not match input type StringType")
    }
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
}



defined trait MyParams


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [33]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.{StructField, StructType, DoubleType, StringType}
import org.apache.spark.sql.functions.{col, udf}

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap) = {
    defaultCopy(extra)
  }
    
    override def transformSchema(schema: StructType): StructType = {
    val idx1 = schema.fieldIndex($(inputCol1))
    val field1 = schema.fields(idx1)
    val idx2 = schema.fieldIndex($(inputCol2))
    val field2 = schema.fields(idx2)
    if (field1.dataType != StringType && field2.dataType != StringType) {
      throw new Exception(s"Input type ${field1.dataType} or ${field2.dataType} did not match input type StringType")
    }
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        import dataset.sparkSession.implicits._
     dataset.select(col("*"),(dataset($(inputCol1))/dataset($(inputCol2))).as($(outputCol)))
  }
}

defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [34]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

+-------------------+
|rooms_per_household|
+-------------------+
|  6.984126984126984|
|  6.238137082601054|
|  8.288135593220339|
| 5.8173515981735155|
|  6.281853281853282|
+-------------------+
only showing top 5 rows



myTransformer = configurablewordcount_dda2037ba372


myDataset: Unit = ()


configurablewordcount_dda2037ba372

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [35]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

defined class Instance
defined object Helper


In [36]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

defined class LR


In [37]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

defined class MyLinearModel
defined class MyLinearModelImpl


In [38]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

defined class MyLinearRegression
defined class MyLinearRegressionImpl


In [39]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Training
+--------------------+--------+--------------------+
|          prediction|   label|            features|
+--------------------+--------+--------------------+
|-8.68122874921077...|103600.0|[-62.040445150874...|
|-8.66412244270375...|111400.0|[-62.020480405854...|
|-8.66541024687954...| 76100.0|[-62.015489219599...|
|-8.66428167361475...|106700.0|[-62.005506847089...|
|-8.66580468048986...| 50800.0|[-62.005506847089...|
+--------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 8.2870388499168725E37


lr = linReg_5f763f76a470
model = linReg_5f763f76a470
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_f8828e4ebc6d
rmse = 8.2870388499168725E37


8.2870388499168725E37

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

### Load the data

In [71]:
val credit_cards = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/ccdefault.csv")

credit_cards = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [72]:
//rename label as target
val labeled_cards = credit_cards.withColumnRenamed("DEFAULT","LABEL")

labeled_cards = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

### Explanatory analysis
At the beginning we want to see the features in the dataset and their types in order to identify both numerical attributes and categorical attributes. Also, inspect the dataset to find possible missing values.

In [73]:
//in the schema there is no string input, therefore we don't need any StringIndexer.
labeled_cards.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- LABEL: integer (nullable = true)

In [74]:
//no missing values in the dataset, therefore we don't need to use an Imputer.
for (c <- labeled_cards.columns) {
    print(c + " ")
    println(labeled_cards.select(c).where(c+" IS NULL").count())
}

ID 0
LIMIT_BAL 0
SEX 0
EDUCATION 0
MARRIAGE 0
AGE 0
PAY_0 0
PAY_2 0
PAY_3 0
PAY_4 0
PAY_5 0
PAY_6 0
BILL_AMT1 0
BILL_AMT2 0
BILL_AMT3 0
BILL_AMT4 0
BILL_AMT5 0
BILL_AMT6 0
PAY_AMT1 0
PAY_AMT2 0
PAY_AMT3 0
PAY_AMT4 0
PAY_AMT5 0
PAY_AMT6 0
LABEL 0


In [75]:
labeled_cards.count()

30000

In [76]:
%%dataframe
labeled_cards

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,LABEL
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800,0
7,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
8,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542,0
9,140000,2,3,1,28,0,0,2,0,0,0,11285,14096,12108,12211,11793,3719,3329,0,432,1000,1000,1000,0
10,20000,1,3,2,35,-2,-2,-2,-2,-1,-1,0,0,0,0,13007,13912,0,0,0,13007,1122,0,0


PAY: repayment status is: -1 = pay duly; 1 = payment delay for one month etc.
BILL_AMT: amount of bill statement (NT dollar)
PAY_AMT: amount of previous payment (NT dollar)

It is fundamental to get statistics about data. In particular, we want to count the number of instances that have a certain attribute, the mean of that attribute, the standard deviation and both the minimum and maximum values.

In [77]:
//shows dataset information
labeled_cards.select("sex", "marriage", "age", "education","limit_bal").describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|               sex|          marriage|              age|         education|         limit_bal|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|             30000|             30000|            30000|             30000|             30000|
|   mean|1.6037333333333332|1.5518666666666667|          35.4855|1.8531333333333333|167484.32266666667|
| stddev|0.4891291960902602|0.5219696006132467|9.217904068090155|0.7903486597207269|129747.66156720246|
|    min|                 1|                 0|               21|                 0|             10000|
|    max|                 2|                 3|               79|                 6|           1000000|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [79]:
//shows dataset further information
labeled_cards.groupBy("label").count().withColumn("percentage",col("count")/30000).show()
labeled_cards.groupBy("sex").count().withColumn("percentage",col("count")/30000).show()
labeled_cards.groupBy("marriage").count.withColumn("percentage",col("count")/30000).show()
print("Distinct ages: " + labeled_cards.select("age").distinct.count +"\n\n")
labeled_cards.groupBy("education").count.withColumn("percentage",col("count")/30000).show()
print("Distinct credits: "+ labeled_cards.select("limit_bal").distinct.count)

//IMPORTANT: the dataset is highly unbalanced. We need to study F1-score and AUC

+-----+-----+----------+
|label|count|percentage|
+-----+-----+----------+
|    1| 6636|    0.2212|
|    0|23364|    0.7788|
+-----+-----+----------+

+---+-----+-------------------+
|sex|count|         percentage|
+---+-----+-------------------+
|  1|11888|0.39626666666666666|
|  2|18112| 0.6037333333333333|
+---+-----+-------------------+

+--------+-----+--------------------+
|marriage|count|          percentage|
+--------+-----+--------------------+
|       1|13659|              0.4553|
|       3|  323|0.010766666666666667|
|       2|15964|  0.5321333333333333|
|       0|   54|              0.0018|
+--------+-----+--------------------+

Distinct ages: 56

+---------+-----+--------------------+
|education|count|          percentage|
+---------+-----+--------------------+
|        1|10585| 0.35283333333333333|
|        6|   51|              0.0017|
|        3| 4917|              0.1639|
|        5|  280|0.009333333333333334|
|        4|  123|              0.0041|
|        2|14030|  0

### Data cleaning
In this section we identify useless attributes (e.g. the ID of each instance) and we prepare the header of the dataframe by using meaningful names.

In [80]:
//fixes the first PAY column and drops the ID column since not useful.
val new_labeled_cards = labeled_cards.withColumnRenamed("PAY_0", "PAY_1").drop("id")

new_labeled_cards = [LIMIT_BAL: int, SEX: int ... 22 more fields]


[LIMIT_BAL: int, SEX: int ... 22 more fields]

In [81]:
new_labeled_cards.printSchema()

root
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_1: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- LABEL: integer (nullable = true)



### Data augmentation: calculate the ratio between bill and limit
In order to compare different people, we do not have to consider the absolute value of payments but the ratio between payments and balance limit.

In [82]:
val newdf1 = new_labeled_cards.withColumn("PayLimitRatio1", new_labeled_cards("BILL_AMT1") / new_labeled_cards("LIMIT_BAL"))
val newdf2 = newdf1.withColumn("PayLimitRatio2", newdf1("BILL_AMT2") / newdf1("LIMIT_BAL"))
val newdf3 = newdf2.withColumn("PayLimitRatio3", newdf2("BILL_AMT3") / newdf2("LIMIT_BAL"))
val newdf4 = newdf3.withColumn("PayLimitRatio4", newdf3("BILL_AMT4") / newdf3("LIMIT_BAL"))
val newdf5 = newdf4.withColumn("PayLimitRatio5", newdf4("BILL_AMT5") / newdf4("LIMIT_BAL"))
val augmented_df = newdf5.withColumn("PayLimitRatio6", newdf5("BILL_AMT6") / newdf5("LIMIT_BAL"))


newdf1 = [LIMIT_BAL: int, SEX: int ... 23 more fields]
newdf2 = [LIMIT_BAL: int, SEX: int ... 24 more fields]
newdf3 = [LIMIT_BAL: int, SEX: int ... 25 more fields]
newdf4 = [LIMIT_BAL: int, SEX: int ... 26 more fields]
newdf5 = [LIMIT_BAL: int, SEX: int ... 27 more fields]
augmented_df = [LIMIT_BAL: int, SEX: int ... 28 more fields]


[LIMIT_BAL: int, SEX: int ... 28 more fields]

### Further cleaning: eliminate negative values for one-hot encoding
In order to use the one-hot encoding on the categorical data, we have to substitute the negative values with zero values.

In [83]:
val newdf7 = augmented_df.withColumn("PAY_1_C", when($"PAY_1" <= 0, 0).otherwise($"PAY_1"))
val newdf8 = newdf7.withColumn("PAY_2_C", when($"PAY_2" <= 0, 0).otherwise($"PAY_2"))
val newdf9 = newdf8.withColumn("PAY_3_C", when($"PAY_3" <= 0, 0).otherwise($"PAY_3"))
val newdf10 = newdf9.withColumn("PAY_4_C", when($"PAY_4" <= 0, 0).otherwise($"PAY_4"))
val newdf11 = newdf10.withColumn("PAY_5_C", when($"PAY_5" <= 0, 0).otherwise($"PAY_5"))
val final_df = newdf11.withColumn("PAY_6_C", when($"PAY_6" <= 0, 0).otherwise($"PAY_6"))

newdf7 = [LIMIT_BAL: int, SEX: int ... 29 more fields]
newdf8 = [LIMIT_BAL: int, SEX: int ... 30 more fields]
newdf9 = [LIMIT_BAL: int, SEX: int ... 31 more fields]
newdf10 = [LIMIT_BAL: int, SEX: int ... 32 more fields]
newdf11 = [LIMIT_BAL: int, SEX: int ... 33 more fields]
final_df = [LIMIT_BAL: int, SEX: int ... 34 more fields]


[LIMIT_BAL: int, SEX: int ... 34 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (sex,education,marriage,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors.
This step is fundamental in order to normalize the numerical attributes and transform into categories the categorical attributes.

In [84]:
// label columns
val colLabel = "LABEL"

// categorical columns
val colCat1 = "PAY_1_C"
val colCat2 = "PAY_2_C"
val colCat3 = "PAY_3_C"
val colCat4 = "PAY_4_C"
val colCat5 = "PAY_5_C"
val colCat6 = "PAY_6_C"
val colCat7 = "SEX"
val colCat8 = "EDUCATION"
val colCat9 = "MARRIAGE"

// numerical columns
val colNum = final_df.columns.filter(_ != colLabel).filter(_ != colCat1).filter(_ != colCat2).filter(_ != colCat3).filter(_ != colCat4).filter(_ != colCat5).filter(_ != colCat6).filter(_ != colCat7).filter(_ != colCat8).filter(_ != colCat9)

colLabel = LABEL
colCat1 = PAY_1_C
colCat2 = PAY_2_C
colCat3 = PAY_3_C
colCat4 = PAY_4_C
colCat5 = PAY_5_C
colCat6 = PAY_6_C
colCat7 = SEX
colCat8 = EDUCATION
colCat9 = MARRIAGE
colNum = Array(LIMIT_BAL, AGE, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, PayLimitRatio1, PayLimitRatio2, PayLimitRatio3, PayLimitRatio4, PayLimitRatio5, PayLimitRatio6)


Array(LIMIT_BAL, AGE, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, PayLimitRatio1, PayLimitRatio2, PayLimitRatio3, PayLimitRatio4, PayLimitRatio5, PayLimitRatio6)

### Scaling
With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the cards data: the total number of limit_bal ranges from  10000 to 1000000, while the age only range from 21 to 79. 

To get all attributes to have the same scale we use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we use the `StandardScaler` Estimator. To use `StandardScaler`, we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vector.

In [85]:
val vectorAssembler = new VectorAssembler().setInputCols(colNum).setOutputCol("feature_scaling")
//val featured_cards = vectorAssembler.transform(augmented_df)

val standardScaler = new StandardScaler().setInputCol("feature_scaling").setOutputCol("scaled_features")
//val scaled_cards = scaler.fit(featured_cards).transform(featured_cards)

//scaled_cards.show(5)

vectorAssembler = vecAssembler_505f56778b1c
standardScaler = stdScal_77e76376f34d


stdScal_77e76376f34d

### One-hot encoding of categorical data

In [86]:
val encoder = new OneHotEncoderEstimator().setInputCols(Array(colCat1,colCat2,colCat3,colCat4,colCat5,colCat6,colCat7,colCat8,colCat9)).setOutputCols(Array("PAY_1_e", "PAY_2_e", "PAY_3_e", "PAY_4_e", 
                 "PAY_5_e", "PAY_6_e","SEX_e", "EDUCATION_e", "MARRIAGE_e"))
//val data_encoded = nice_encoder.fit(augmented_df).transform(augmented_df)

encoder = oneHotEncoder_a873fd3f6c80


oneHotEncoder_a873fd3f6c80

### Define a pipeline
The transformations applied both on the numerical attributes and the categorical attributes are managed through a pipeline. This makes the code more readable and more maintanable for future works on this project.

In [87]:
val numPipeline = new Pipeline().setStages(Array(vectorAssembler, standardScaler))
val catPipeline = new Pipeline().setStages(Array(encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))

val pipelined = pipeline.fit(final_df).transform(final_df)

numPipeline = pipeline_fba112c5027d
catPipeline = pipeline_156db07e3748
pipeline = pipeline_1381502e6076
pipelined = [LIMIT_BAL: int, SEX: int ... 45 more fields]


[LIMIT_BAL: int, SEX: int ... 45 more fields]

Now, use VectorAssembler to put all attributes of the final dataset cards into a big vector, and call the new column features. The goal of this operation is to one one macro input column, known as features, and one target column, in this case called label.

In [88]:
//Concatenate numerical attributes and categorical attributes to create a unique dataset

val new_vectorAssembler = new VectorAssembler().setInputCols(Array("scaled_features", "PAY_1_e", "PAY_2_e", "PAY_3_e", "PAY_4_e", 
                 "PAY_5_e", "PAY_6_e","SEX_e", "EDUCATION_e", "MARRIAGE_e")).setOutputCol("features")
val dataset = new_vectorAssembler.transform(pipelined).select("features", "label")

dataset.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(85,[0,1,2,3,4,5,...|    1|
|(85,[0,1,2,3,7,8,...|    1|
|(85,[0,1,8,9,10,1...|    0|
|(85,[0,1,8,9,10,1...|    0|
|(85,[0,1,2,4,8,9,...|    0|
+--------------------+-----+
only showing top 5 rows



new_vectorAssembler = vecAssembler_f2571d349bee
dataset = [features: vector, label: int]


[features: vector, label: int]

### Models
Here we going to make four different classification models:

Logistic regression
Decision tree 
Random forest
But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (trainSet) with 80% of the whole data, and test dataset (testSet) with 20% of it.

In [89]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8,0.2))

trainSet = [features: vector, label: int]
testSet = [features: vector, label: int]


[features: vector, label: int]

### Logistic Regression
#### Training and quick evaluation on test set for Logistic Regression

In [132]:
import org.apache.spark.ml.classification.LogisticRegression

// train the model
val lr = new LogisticRegression().setFeaturesCol("features").setLabelCol("label")
val lrModel = lr.fit(trainSet)

val lrtrainingSummary = lrModel.summary
println(s"Coefficients: " + lrModel.coefficients + s", Intercept: " + lrModel.intercept)

// Print the training summary and the history
val lrtrainingBinarySummary = lrModel.binarySummary
val lrobjectiveHistory = lrtrainingBinarySummary.objectiveHistory
val roc = lrtrainingBinarySummary.roc
roc.show()
println(s"Area Under ROC (AUC) at the end of training: ${lrtrainingBinarySummary.areaUnderROC}")

//Compute predictions of the test set
val lrPredictions = lrModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)
val resultslr = lrPredictions.select("features", "label", "prediction")

//Evaluate results
val lrEvaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val lrAuc = lrEvaluator.evaluate(lrPredictions)
println(s"Area Under ROC (AUC) on test data before tuning= "+lrCvAuc)

Coefficients: [-0.21251085806798672,0.04980379710379821,-0.3106655499974538,0.19367906603172116,-0.017188711797854746,-0.035101328414280444,-0.008351252542037299,-0.2030914585445207,0.03149457708629808,0.12461803272649173,0.049897914954507905,-0.05144092386964936,0.048431585088152034,-0.09658254202637453,-0.18522544588940443,-0.14200027637844975,-0.008573401768719412,-0.042548337452558524,-0.029458974051090613,-0.056678531842456095,-0.1216843653297743,0.09617550579655276,0.016490488795684224,0.0015559691117179115,-0.021795114920313132,0.04879320959759785,-0.8625119213708119,0.4032873284098105,1.9122733748215843,2.1494392109805336,2.1437975978836783,1.541341140211352,0.5926379975812269,4.310411509937659,0.4594745402780495,-0.33747947463372,0.18188684867754937,0.06604738444339056,-0.8308487063204523,1.3736099742621035,8.8395688601006,6.4114268711557045,-0.0918969634471387,-6.014597253895815,0.2547088384440449,0.42046160243008673,0.034243952056200684,-0.8197126871474605,-3.398604070255924

lr = logreg_0e8c9a36d093
lrModel = LogisticRegressionModel: uid = logreg_0e8c9a36d093, numClasses = 2, numFeatures = 85
lrtrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@11968845
lrtrainingBinarySummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@11968845
lrobjectiveHistory = Array(0.5306850506811015, 0.48636835446952237, 0.46698509155449297, 0.46273725373401514, 0.457676...


Array(0.5306850506811015, 0.48636835446952237, 0.46698509155449297, 0.46273725373401514, 0.457676...

#### Hyperparameter tuning for Logistic Regression

In [131]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

//K-fold cross validation
val lrParamGrid = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(0,0.001, 0.01, 0.1)).addGrid(lr.elasticNetParam, Array(0, 0.0001, 0.001, 0.01))
    .build()

val lrCv = new CrossValidator()
    .setEstimator(lr)
    .setEvaluator(lrEvaluator)
    .setEstimatorParamMaps(lrParamGrid)
    .setNumFolds(4)

//Fit the best model on the training data
val lrCvModel = lrCv.fit(dataset)

//Compute predictions on the test set
val lrCvPredictions = lrCvModel.transform(testSet)
lrCvPredictions.select("prediction", "label", "features").show(5)
val resultslrCv = lrCvPredictions.select("features", "label", "prediction")

val lrCvAuc = lrEvaluator.evaluate(lrCvPredictions)
println(s"Area Under ROC (AUC) on test data after tuning= "+lrCvAuc)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data after tuning= 0.661137805549758


lrParamGrid = 


Array({
	logreg_c695c27d1ade-elasticNetParam: 0.0,
	logreg_c695c27d1ade-regParam: 0.0
}, {
	logreg_c695c27d1ade-elasticNetParam: 1.0E-4,
	logreg_c695c27d1ade-regParam: 0.0
}, {
	logreg_c695c27d1ade-elasticNetParam: 0.001,
	logreg_c695c27d1ade-regParam: 0.0
}, {
	logreg_c695c27d1ade-elasticNetParam: 0.01,
	logreg_c695c27d1ade-regParam: 0.0
}, {
	logreg_c695c27d1ade-elasticNetParam: 0.0,
	logreg_c695c27d1ade-regParam: 0.001
}, {
	logreg_c695c27d1ade-elasticNetParam: 1.0E-4,
	logreg_c695c27d1ade-regParam: 0.001
}, {
	logreg_c695c27d1ade-elasticNetParam: 0.001,
	logreg_...


### Decision Tree 
#### Training and quick evaluation on test set for Decision Tree

In [133]:
import org.apache.spark.ml.classification.DecisionTreeClassifier

//Train the model
val dt = new DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("label")
val dtModel = dt.fit(trainSet)

//Make predictions on test set
val dtPredictions = dtModel.transform(testSet)
dtPredictions.select("prediction", "rawPrediction", "label", "features").show(5)

val resultsdt = dtPredictions.select("features", "label", "prediction")

val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")//.setMetricName("accuracy")
val accuracy = evaluator.evaluate(dtPredictions)
println(s"Area Under ROC (AUC) on test data before tuning= "+accuracy)

+----------+----------------+-----+--------------------+
|prediction|   rawPrediction|label|            features|
+----------+----------------+-----+--------------------+
|       0.0|[16734.0,2836.0]|    0|(85,[0,1,2,3,4,5,...|
|       0.0|[16734.0,2836.0]|    1|(85,[0,1,2,3,4,5,...|
|       0.0|[16734.0,2836.0]|    0|(85,[0,1,2,3,4,5,...|
|       0.0|[16734.0,2836.0]|    0|(85,[0,1,2,3,4,5,...|
|       0.0|[16734.0,2836.0]|    1|(85,[0,1,2,3,4,5,...|
+----------+----------------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data before tuning= 0.6623450183835934


dt = dtc_472d38cc3ff3
dtModel = DecisionTreeClassificationModel (uid=dtc_472d38cc3ff3) of depth 5 with 35 nodes
dtPredictions = [features: vector, label: int ... 3 more fields]
resultsdt = [features: vector, label: int ... 1 more field]
evaluator = binEval_80bb76b30141
accuracy = 0.6623450183835934


0.6623450183835934

#### Hyperparameter tuning phase for Decision Tree

In [134]:
val dtParamGrid = new ParamGridBuilder().addGrid(dt.maxBins, Array(20,40,80)).addGrid(dt.maxDepth, Array(5,10,20)).build()

val dtCv = new CrossValidator().setEstimator(dt).setEvaluator(evaluator).setEstimatorParamMaps(dtParamGrid).setNumFolds(4)

val dtCvModel = dtCv.fit(dataset)

val dtCvPredictions = dtCvModel.transform(testSet)
dtCvPredictions.select("prediction", "label", "features").show(5)

val resultsdtCv = dtCvPredictions.select("features", "label", "prediction")

val dtCvAuc = evaluator.evaluate(dtCvPredictions)
println(s"Area Under ROC (AUC) on test data after tuning= "+dtCvAuc)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data after tuning= 0.6539625549036254


dtParamGrid = 
dtCv = cv_25f3f6546eec
dtCvModel = cv_25f3f...


Array({
	dtc_472d38cc3ff3-maxBins: 20,
	dtc_472d38cc3ff3-maxDepth: 5
}, {
	dtc_472d38cc3ff3-maxBins: 40,
	dtc_472d38cc3ff3-maxDepth: 5
}, {
	dtc_472d38cc3ff3-maxBins: 80,
	dtc_472d38cc3ff3-maxDepth: 5
}, {
	dtc_472d38cc3ff3-maxBins: 20,
	dtc_472d38cc3ff3-maxDepth: 10
}, {
	dtc_472d38cc3ff3-maxBins: 40,
	dtc_472d38cc3ff3-maxDepth: 10
}, {
	dtc_472d38cc3ff3-maxBins: 80,
	dtc_472d38cc3ff3-maxDepth: 10
}, {
	dtc_472d38cc3ff3-maxBins: 20,
	dtc_472d38cc3ff3-maxDepth: 20
}, {
	dtc_472d38cc3ff3-maxBins: 40,
	dtc_472d38cc3ff3-maxDepth: 20
}, {
	dtc_472d38cc3ff3-maxBins: 80,
	dtc_472d38cc3ff3-maxDepth: 20
})


cv_25f3f...

### Random Forest
#### Training and quick evaluation on test set for Random Forest

In [130]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}


val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val rfPredictions = rfModel.transform(testSet)
rfPredictions.select("prediction", "label", "features").show(5)
val resultsrf=rfPredictions.select("prediction", "label", "features")

// select (prediction, true label) and compute test error
val rfEvaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val accuracy = evaluator.evaluate(rfPredictions)
println(s"Area Under ROC (AUC) on test data before tuning= "+accuracy)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    0|(85,[0,1,2,3,4,5,...|
|       0.0|    1|(85,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data before tuning= 0.6596203548669053


rf = rfc_f1d19f953c48
rfModel = RandomForestClassificationModel (uid=rfc_f1d19f953c48) with 20 trees
rfPredictions = [features: vector, label: int ... 3 more fields]
resultsrf = [prediction: double, label: int ... 1 more field]
rfEvaluator = binEval_ffcd41d048a1
accuracy = 0.6596203548669053


lastException: Throwable = null


0.6596203548669053

#### Hyperparameter tuning phase for Random Forest

In [ ]:
val rfParamGrid = new ParamGridBuilder().addGrid(rf.numTrees, Array(2,5,15)).addGrid(rf.maxDepth, Array(5,10,20)).addGrid(rf.maxBins, Array(20,40,80)).build()

val rfCv = new CrossValidator().setEstimator(rf).setEvaluator(rfEvaluator).setEstimatorParamMaps(rfParamGrid).setNumFolds(4)

val rfCvModel = rfCv.fit(dataset)

val rfCvPredictions = rfCvModel.transform(testSet)
rfCvPredictions.select("prediction", "label", "features").show(5)
val resultsrfCv=rfCvPredictions.select("prediction", "label", "features")

val rfAuc = evaluator.evaluate(rfCvPredictions)
println(s"Area Under ROC (AUC) on test data after tuning= $rfAuc")

### Further evaluations

In [125]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics 

//F1-score and AU P-R curve can be computed for each of the 6 models, choosing among these variables:
//resultslrCv
//resultsdt
//resultsdtCv
//resultsrf
//resultsrfCv
val predictionAndLabels = resultsrf.select($"prediction",$"label").as[(Double, Double)].rdd

predictionAndLabels = MapPartitionsRDD[15145] at rdd at <console>:70


MapPartitionsRDD[15145] at rdd at <console>:70

In [126]:
//instantiate new metrics objects
val bMetrics = new BinaryClassificationMetrics(predictionAndLabels)
val mMetrics = new MulticlassMetrics(predictionAndLabels)
val labels = mMetrics.labels
//prints out the Confusion matrix
println("Confusion matrix:")
println(mMetrics.confusionMatrix)

Confusion matrix:
4592.0  243.0  
831.0   487.0  


bMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@53c9a12b
mMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@616d4d6e
labels = Array(0.0, 1.0)


Array(0.0, 1.0)

In [127]:
//precision by label
labels.foreach { l => println(s"Precision($l) = " + mMetrics.precision(l))}
//recall by label
labels.foreach { l => println(s"Recall($l) = " + mMetrics.recall(l))}
//false positive rate by label
labels.foreach { l =>println(s"FPR($l) = " + mMetrics.falsePositiveRate(l))}
//F-measure by label
labels.foreach { l =>println(s"F1-Score($l) = " + mMetrics.fMeasure(l))}

Precision(0.0) = 0.8467637838834593
Precision(1.0) = 0.6671232876712329
Recall(0.0) = 0.949741468459152
Recall(1.0) = 0.36949924127465855
FPR(0.0) = 0.6305007587253414
FPR(1.0) = 0.05025853154084799
F1-Score(0.0) = 0.895301228309612
F1-Score(1.0) = 0.47558593749999994


In [128]:
// precision by threshold
val precision = bMetrics.precisionByThreshold
precision.foreach { case (t, p) =>println(s"Threshold: $t, Precision: $p")}
// pecall by threshold
val recall = bMetrics.recallByThreshold
recall.foreach { case (t, r) =>println(s"Threshold: $t, Recall: $r")}
// Precision-Recall Curve
val PRC = bMetrics.pr

// F-measure
val f1Score = bMetrics.fMeasureByThreshold
f1Score.foreach { case (t, f) =>println(s"Threshold: $t, F-score: $f, Beta = 1")}
val beta = 0.5
val fScore = bMetrics.fMeasureByThreshold(beta)
f1Score.foreach { case (t, f) =>println(s"Threshold: $t, F-score: $f, Beta = 0.5")}
// AUPRC
val auPRC = bMetrics.areaUnderPR
println("Area under precision-recall curve = " + auPRC)

// Compute thresholds used in ROC and PR curves
val thresholds = precision.map(_._1)

// ROC Curve
val roc = bMetrics.roc

// AUROC
val auROC = bMetrics.areaUnderROC
println("Area under ROC = " + auROC)

Area under precision-recall curve = 0.5243404532560315
Area under ROC = 0.6596203548669053


precision = MapPartitionsRDD[15162] at map at BinaryClassificationMetrics.scala:217
recall = MapPartitionsRDD[15163] at map at BinaryClassificationMetrics.scala:217
PRC = UnionRDD[15166] at union at BinaryClassificationMetrics.scala:110
f1Score = MapPartitionsRDD[15167] at map at BinaryClassificationMetrics.scala:217
beta = 0.5
fScore = MapPartitionsRDD[15168] at map at BinaryClassificationMetrics.scala:217
auPRC = 0.5243404532560315
thresholds = MapPartitionsRDD[15174] at map at <console>:90


roc: org.apache.spark.rdd.RDD...


MapPartitionsRDD[15174] at map at <console>:90